In [2]:
VERSION = 4

CONSTRAINT_TOOLS = '/scratch/ucgd/lustre-work/quinlan/u6018199/constraint-tools'
CONSTRAINT_TOOLS_DATA = '/scratch/ucgd/lustre-work/quinlan/data-shared/constraint-tools'

NUMBER_NEGATIVE_EXAMPLES = 1000
LABELED_REGIONS = f'labeled-regions.{VERSION}.bed'

import sys
sys.path.append(f'{CONSTRAINT_TOOLS}/utilities')

## Make large windows

In [3]:
from shell import shell 

LARGE_WINDOW_FILENAME = f'{CONSTRAINT_TOOLS_DATA}/benchmark-genome-wide-predictions/chen-et-al-2022/large-windows.bed'

def make_large_windows(): 
  chromosome_sizes = f'{CONSTRAINT_TOOLS_DATA}/reference/grch38/chromosome-sizes/hg38.chrom.sizes.sorted'
  large_window_size = 10000
  cmd = f'bedtools makewindows -g {chromosome_sizes} -w {large_window_size} > {LARGE_WINDOW_FILENAME}'    
  print(shell(cmd))

make_large_windows()

In [13]:
import pandas as pd 

# intersect large windows with deletions (both het and homalt), and filter
def read_large_windows_intersect_all_deletions():
  cmd = f'bash {CONSTRAINT_TOOLS}/experiments/germline-model/chen-et-al-2022/intersect-large-windows-with-topmed-deletions-and-filter.sh'
  print(shell(cmd))
  return 
  df = pd.read_csv(
    f"{CONSTRAINT_TOOLS_DATA}/benchmark-genome-wide-predictions/chen-et-al-2022/large-windows-with-deletion-counts.bed", 
    sep = '\t',
  )  
  return df 
  df = df[
    (df['chrom'] != 'chrX') &
    (df['chrom'] != 'chrY')
  ]
  return df 
  group_columns = [
    'chrom',
    'start',
    'end',
    'truly constrained',
    'tag',
    'min negative_new_chen_score_window',
    'count negative_new_chen_score_window',
    'region_id'
  ]
  aggregation_functions = {
    'SVLEN': [custom_max],
    'SVTYPE': [collapse, count],
    # 'SV_ID': [custom_list],
    'Het': [custom_max],
    'HomAlt': [custom_max],
    # 'region-deletion-overlap': [custom_max]
  }
  aggregation_columns = list(aggregation_functions.keys())
  new_columns = group_columns + aggregation_columns
  df = df[new_columns]
  df = aggregate_over_deletions(df, group_columns, aggregation_functions)
  df = df.rename(columns={
    'chrom': 'chrom_region',
    'start': 'start_region',
    'end': 'end_region',
  })
  df['region is deleted in topmed'] = df['collapse SVTYPE'] == 'DEL'
  print('number of regions that are deleted in TopMed (True) or not (False):')
  print(df['region is deleted in topmed'].value_counts())
  return df   
  
pd.set_option('display.max_rows', 1000)

read_large_windows_intersect_all_deletions()

Exception: Error: line number 45 of file /dev/fd/62 has 1 fields, but 3 were expected.

## Get a set of regions known to be constrained, and a set of regions that are assumed to be constrained, together with their Chen scores 

In [10]:
import pandas as pd

def read_disease_enhancers_intersect_chen_windows():
  # download-process-data/khurana/README.md
  df = pd.read_csv(
    f"{CONSTRAINT_TOOLS_DATA}/khurana/disease-enhancers-intersect-chen-windows.bed", 
    sep = '\t',
    names = [
        'chrom_enhancer', 'start_enhancer', 'end_enhancer', 
        'disease', 'enhancer_predicted_LoF_tolerance_prob', 'enhancer_predicted_LoF_tolerance_status', 
        'chrom_window', 'start_window', 'end_window', 
        'new_chen_score_window'
    ]
  )
  df = df.drop_duplicates() 
  df['negative_new_chen_score_window'] = -df['new_chen_score_window']
  df = df.drop(columns=[
    'new_chen_score_window',
    'chrom_window', 'start_window', 'end_window',
    'disease', 'enhancer_predicted_LoF_tolerance_prob', 'enhancer_predicted_LoF_tolerance_status'
  ])
  df = df.rename(columns={
    'chrom_enhancer': 'chrom', 
    'start_enhancer': 'start', 
    'end_enhancer': 'end'
  })
  df['truly constrained'] = True
  df['tag'] = 'disease_enhancers'
  return df 

read_disease_enhancers_intersect_chen_windows()

,chrom,start,end,negative_new_chen_score_window,truly constrained,tag
0,chr1,21346107,21348107,-3.432213,True,disease_enhancers
1,chr1,93909844,93911044,-2.645724,True,disease_enhancers
2,chr1,155293209,155293609,-4.190955,True,disease_enhancers
3,chr1,160032410,160033810,-2.719339,True,disease_enhancers
4,chr1,160032410,160033810,-0.167476,True,disease_enhancers
...,...,...,...,...,...,...
116,chr20,46114761,46114961,0.241142,True,disease_enhancers
117,chr22,28787612,28789612,2.138137,True,disease_enhancers
118,chr22,28787612,28789612,0.488883,True,disease_enhancers
119,chr22,28787612,28789612,1.762075,True,disease_enhancers


In [11]:
# get low-lof-tolerance enhancers
def read_low_lof_tolerance_enhancers_intersect_chen_windows():
  # download-process-data/khurana/README.md
  df = pd.read_csv(
    f"{CONSTRAINT_TOOLS_DATA}/khurana/low-lof-tolerance-enhancers-intersect-chen-windows.bed", 
    sep = '\t',
    names = [
        'chrom_enhancer_hg38', 'start_enhancer_hg38', 'end_enhancer_hg38', 
        'enhancer_hg19', 'unknown',
        'chrom_window', 'start_window', 'end_window', 
        'new_chen_score_window'
    ]
  )
  df = df.drop_duplicates() 
  df['negative_new_chen_score_window'] = -df['new_chen_score_window']
  df = df.drop(columns=[
    'new_chen_score_window',
    'chrom_window', 'start_window', 'end_window',
    'enhancer_hg19', 'unknown'
  ])
  df = df.rename(columns={
    'chrom_enhancer_hg38': 'chrom', 
    'start_enhancer_hg38': 'start', 
    'end_enhancer_hg38': 'end'
  })
  df['truly constrained'] = True
  df['tag'] = 'low_lof_tolerance'
  return df 

read_low_lof_tolerance_enhancers_intersect_chen_windows().iloc[:10]

,chrom,start,end,negative_new_chen_score_window,truly constrained,tag
0,chr1,61587728,61589928,-0.884984,True,low_lof_tolerance
1,chr1,61587728,61589928,-3.667483,True,low_lof_tolerance
2,chr1,61587728,61589928,-1.787719,True,low_lof_tolerance
3,chr1,87355917,87357117,-1.174601,True,low_lof_tolerance
4,chr1,87355917,87357117,-2.971624,True,low_lof_tolerance
5,chr1,87355917,87357117,-1.597840,True,low_lof_tolerance
9,chr1,169941659,169943659,-0.732262,True,low_lof_tolerance
10,chr1,169941659,169943659,-0.494991,True,low_lof_tolerance
11,chr2,66069666,66071266,-1.628703,True,low_lof_tolerance
12,chr2,66069666,66071266,-1.923346,True,low_lof_tolerance


In [12]:
def aggregate_over_windows(df): 
  columns = df.columns.tolist()
  group_columns = [column for column in columns if column != 'negative_new_chen_score_window']
  groups = df.groupby(group_columns)
  aggregation_functions = {'negative_new_chen_score_window': ['min', 'count']}
  aggregated = groups.agg(aggregation_functions)  
  df = aggregated.reset_index()
  df.columns = [' '.join(col[::-1]).strip() for col in df.columns.values]
  return df

aggregate_over_windows(read_low_lof_tolerance_enhancers_intersect_chen_windows())

,chrom,start,end,truly constrained,tag,min negative_new_chen_score_window,count negative_new_chen_score_window
0,chr1,61587728,61589928,True,low_lof_tolerance,-3.667483,3
1,chr1,87355917,87357117,True,low_lof_tolerance,-2.971624,3
2,chr1,169941659,169943659,True,low_lof_tolerance,-0.732262,2
3,chr10,100612643,100615043,True,low_lof_tolerance,-1.667674,4
4,chr10,100686843,100689243,True,low_lof_tolerance,-1.212765,4
5,chr10,100786643,100789043,True,low_lof_tolerance,-2.482828,4
6,chr10,101322243,101324043,True,low_lof_tolerance,-2.900722,2
7,chr10,101505843,101508043,True,low_lof_tolerance,-0.350437,2
8,chr10,101767243,101769443,True,low_lof_tolerance,-3.530818,3
9,chr10,117966089,117967689,True,low_lof_tolerance,-0.709407,2


In [14]:
# TODO 
def get_noncoding_nonenhancer_windows():
  # this bed file has already been aggregated to the window level: 
  # experiments/germline-model/chen-et-al-2022/enhancer-characteristics-enrichment.ipynb 
  filename = f'{CONSTRAINT_TOOLS_DATA}/benchmark-genome-wide-predictions/chen-et-al-2022/enhancer-characteristics-enrichment.bed'
  df = pd.read_csv(filename, sep='\t')
  return df 

  df = df[df['window overlaps merged_exon'] == False] # noncoding
  df = df[df['window overlaps enhancer'] == False] # nonenhancer

  df = df.drop_duplicates() 
  df = df[[
    'chromosome', 'start', 'end',
    'negative new chen zscore'
  ]]
  df = df.rename(columns={
    'chromosome': 'chrom', 
    'negative new chen zscore': 'min negative_new_chen_score_window'
  }) # type: ignore
  df = df.sample(NUMBER_NEGATIVE_EXAMPLES, random_state=42) # assume most such windows are nonconstrained
  df['truly constrained'] = False
  df['tag'] = 'random_noncoding_nonenhancer'
  df['count negative_new_chen_score_window'] = 1
  return df

pd.set_option('display.max_columns', 100)

get_noncoding_nonenhancer_windows()

,chromosome,start,end,N_observed,N_bar_3_noncoding,N_mean_null_3_noncoding,N_variance_null_3_noncoding,cpg_island overlap,enhancer overlap,merged_exon overlap,window overlaps enhancer,window overlaps merged_exon,window overlaps cpg_island,new chen zscore,negative new chen zscore,eliteness_of_enhancer max,enhancer count,window_enhancer_overlap_bps sum,optimal_gene_targeted_by_enhancer <lambda>,optimal_enhancer_gene_association_score max,eliteness_of_optimal_enhancer_gene_association max
0,chr1,1432000,1433000,260,-0.247615,263.528599,203.072610,NaN,1000.0,NaN,True,False,False,4.299894,-4.299894,1.0,1,1000,VWA1,758.12,1.0
1,chr1,1435000,1436000,275,-7.547130,387.939429,223.937877,440.0,1000.0,444.0,True,True,True,3.331645,-3.331645,1.0,1,1000,VWA1,758.12,1.0
2,chr1,1449000,1450000,225,0.258359,221.559138,177.372370,NaN,NaN,311.0,False,True,False,1.817267,-1.817267,NaN,0,0,.,NaN,NaN
3,chr1,1450000,1451000,313,2.197534,281.968822,199.400099,NaN,NaN,758.0,False,True,False,1.365863,-1.365863,NaN,0,0,.,NaN,NaN
4,chr1,1451000,1452000,293,2.367303,260.131776,192.772432,NaN,NaN,NaN,False,False,False,0.666316,-0.666316,NaN,0,0,.,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1785815,chr9,137269000,137270000,210,-3.445370,257.657992,191.337298,NaN,1000.0,NaN,True,False,False,5.276351,-5.276351,1.0,1,1000,HSALNG0075385,750.64,1.0
1785816,chr9,137275000,137276000,296,1.921453,268.932391,198.445131,NaN,712.0,NaN,True,False,False,2.687348,-2.687348,1.0,2,712,"NELFB,EHMT1",762.48,1.0
1785817,chr9,137282000,137283000,211,-1.919711,237.194349,186.184553,NaN,572.0,641.0,True,True,False,-0.090791,0.090791,0.0,1,572,EXD3,11.79,0.0
1785818,chr9,137290000,137291000,224,-0.682673,233.338243,187.113535,NaN,NaN,NaN,False,False,False,2.162811,-2.162811,NaN,0,0,.,NaN,NaN


In [ ]:
def check_regions_non_overlapping():
  count_self_intersections = (
    f'bedtools intersect'
    f" -a <(tail -n +2 {CONSTRAINT_TOOLS_DATA}/khurana/{LABELED_REGIONS})"
    f" -b <(tail -n +2 {CONSTRAINT_TOOLS_DATA}/khurana/{LABELED_REGIONS})"
    f' -wao'
    f' | wc -l'
  )
  number_self_intersections = shell(count_self_intersections)
  count_number_lines = f'tail -n +2 {CONSTRAINT_TOOLS_DATA}/khurana/{LABELED_REGIONS} | wc -l'
  number_lines = shell(count_number_lines)
  if number_self_intersections != number_lines:
    raise Exception(f'number of self-intersections ({number_self_intersections}) != number of lines ({number_lines})')
  else: 
    print(f'regions ({number_self_intersections}) are non-overlapping')

def create_and_save_labeled_regions(): 
  positive_examples = pd.concat([    
    aggregate_over_windows(read_disease_enhancers_intersect_chen_windows()),
    aggregate_over_windows(read_low_lof_tolerance_enhancers_intersect_chen_windows())
  ])

  negative_examples = get_noncoding_nonenhancer_windows()

  df = pd.concat([positive_examples, negative_examples])

  df = df.reset_index(drop=True) # create new index and drop old index
  df = df.reset_index(drop=False) # make new index into a column
  df = df.rename(columns={'index': 'region_id'})
  new_order = df.columns[1:].tolist() + ['region_id']
  df = df.reindex(columns=new_order)
  
  print('number of regions that are truly constrained (True) or not (False):') 
  print(df['truly constrained'].value_counts())

  df.to_csv(f"{CONSTRAINT_TOOLS_DATA}/khurana/{LABELED_REGIONS}", sep='\t', index=False)

  check_regions_non_overlapping() 

  return df 

pd.set_option('display.max_rows', 10)

create_and_save_labeled_regions()

## Intersect labeled regions with ALL deletions from TopMed 

In [ ]:
import numpy as np 

def collapse(ser): 
  unique_value, = set(ser)
  return unique_value

def count(ser): 
  if len(ser) == 1 and ser.iloc[0] == '.': return 0
  return len(ser) 

def custom_max(ser): 
  if len(ser) == 1 and ser.iloc[0] == '.': return '.'
  return np.max([int(value) for value in list(ser)])

def custom_list(ser): 
  if len(ser) == 1 and ser.iloc[0] == '.': return '.'
  return list(ser)

def aggregate_over_deletions(df, group_columns, aggregation_functions): 
  groups = df.groupby(group_columns)
  aggregated = groups.agg(aggregation_functions)  
  df = aggregated.reset_index()
  df.columns = [' '.join(col[::-1]).strip() for col in df.columns.values]
  return df

def read_labeled_regions_intersect_all_deletions():
  cmd = f'bash {CONSTRAINT_TOOLS}/experiments/germline-model/chen-et-al-2022/intersect-labeled-regions-{VERSION}-with-topmed-deletions.sh {VERSION}'
  print(shell(cmd))
  df = pd.read_csv(
    f"{CONSTRAINT_TOOLS_DATA}/khurana/labeled-regions-{VERSION}-intersect-topmed-deletions.bed", 
    sep = '\t',
  )  
  group_columns = [
    'chrom',
    'start',
    'end',
    'truly constrained',
    'tag',
    'min negative_new_chen_score_window',
    'count negative_new_chen_score_window',
    'region_id'
  ]
  aggregation_functions = {
    'SVLEN': [custom_max],
    'SVTYPE': [collapse, count],
    # 'SV_ID': [custom_list],
    'Het': [custom_max],
    'HomAlt': [custom_max],
    # 'region-deletion-overlap': [custom_max]
  }
  aggregation_columns = list(aggregation_functions.keys())
  new_columns = group_columns + aggregation_columns
  df = df[new_columns]
  df = aggregate_over_deletions(df, group_columns, aggregation_functions)
  df = df.rename(columns={
    'chrom': 'chrom_region',
    'start': 'start_region',
    'end': 'end_region',
  })
  df['region is deleted in topmed'] = df['collapse SVTYPE'] == 'DEL'
  print('number of regions that are deleted in TopMed (True) or not (False):')
  print(df['region is deleted in topmed'].value_counts())
  return df   
  
pd.set_option('display.max_rows', 1000)

REGIONS = read_labeled_regions_intersect_all_deletions()
REGIONS

## Using Chen zscore to predict whether a region is critical or not, and improving that prediction using TopMed deletions

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

# Set the font size
mpl.rcParams['font.size'] = 20

def plot_score_distributions(df, score, xlabel, bins, xlim=None):
  ax = sns.histplot(data=df, x=score, kde=True, bins=bins, hue='truly constrained', stat='count') # stat='density'
  ax.set_xlabel(xlabel)
  ax.set_ylabel('Number of regions')
  if xlim is not None: ax.set_xlim(xlim)
  legend = ax.get_legend()
  legend.set_title('region truly constrained?')
  fig = plt.gcf()
  fig.set_size_inches(10, 5)
  plt.show()

  # Compute the area under the curve for each histogram
  for container in ax.containers:
    print(container.get_label())
    area = 0
    for bar in container.patches:
      area += bar.get_width() * bar.get_height()
    print(f"Area under the curve: {area:.2f}")

plot_score_distributions(REGIONS, score='min negative_new_chen_score_window', xlabel='Chen score of region', bins=40, xlim=(-6, 6))

In [ ]:
from scipy.stats import poisson

def plot_deletion_overlap_distribution(df, regions_class): 
  k = np.array(df['count SVTYPE']) # number of deletions overlapping each region
  poisson_rate_parameter = np.mean(k)

  ser = df['count SVTYPE'].value_counts()
  number_deletions_overlapping_region = np.array(ser.index) # possible (unique) number of deletions overlapping a region
  number_regions = np.array(ser)
  total_number_regions = np.sum(number_regions)
  probability_deletion_overlaps_region = number_regions / total_number_regions

  plt.scatter(number_deletions_overlapping_region, probability_deletion_overlaps_region, label='data')
  plt.scatter(number_deletions_overlapping_region, poisson.pmf(number_deletions_overlapping_region, poisson_rate_parameter), label='Poisson fit')
  plt.yscale('linear') # 'log'
  # plt.ylim([0.0001, 1])
  plt.xlim([0, 20])
  plt.ylim([0, 0.5])
  plt.xlabel('Number of deletions overlapping region')
  plt.ylabel('Probability')
  plt.legend() 
  plt.title(
    f'{regions_class} regions\n'
    f'{poisson_rate_parameter:.2f} deletions overlap each region on average'
  )
  plt.show()

def plot_deletion_overlap_distribution_wrapper():
  constrained = REGIONS['truly constrained']
  plot_deletion_overlap_distribution(REGIONS[constrained == True], regions_class='Constrained')
  plot_deletion_overlap_distribution(REGIONS[constrained == False], regions_class='Unconstrained')

plot_deletion_overlap_distribution_wrapper()

In [ ]:
def predict_constraint_without_deletions(df, zscore_threshold): 
  score = 'min negative_new_chen_score_window'
  df = df[[score, 'truly constrained']].copy()
  df['predicted to be constrained'] = df[score] < zscore_threshold
  return df

predict_constraint_without_deletions(REGIONS, zscore_threshold=-2)

In [ ]:
def predict_constraint_using_deletions(df, zscore_threshold): 
  score = 'min negative_new_chen_score_window'
  df = df[[
    score, 
    'truly constrained', 
    'region is deleted in topmed'
  ]].copy()
  df['predicted to be constrained'] = (
    (df[score] < zscore_threshold) & 
    (df['region is deleted in topmed'] == False)
  )
  return df

pd.set_option('display.max_rows', 10)

predict_constraint_using_deletions(REGIONS, zscore_threshold=-2)

In [ ]:
def compute_precision_recall(df, zscore_threshold, predict_constraint, log=False): 
  df = predict_constraint(df, zscore_threshold)
  
  contingency_table = pd.crosstab(
    df['predicted to be constrained'], 
    df['truly constrained']
  )

  if log:   
    print(zscore_threshold)
    print(contingency_table)

  tp = contingency_table.loc[True, True]
  fp = contingency_table.loc[True, False]
  fn = contingency_table.loc[False, True]
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)

  return precision, recall
  
compute_precision_recall(REGIONS, zscore_threshold=-3, predict_constraint=predict_constraint_without_deletions, log=True)

In [ ]:
compute_precision_recall(REGIONS, zscore_threshold=-3, predict_constraint=predict_constraint_using_deletions, log=True)

In [ ]:
def plot_precision_recall(df, predict_constraint, label): 
  zscore_thresholds = np.arange(-6, 5.5, 0.1) 
  precision_recall = [
    compute_precision_recall(df, zscore_threshold, predict_constraint, log=False)
    for zscore_threshold in zscore_thresholds
  ]
  precisions, recalls = zip(*precision_recall)
  plt.plot(recalls, precisions, label=label)

def plot_precision_recall_wrapper(df): 
  plot_precision_recall(df, predict_constraint_without_deletions, label='without using topmed deletions')
  plot_precision_recall(df, predict_constraint_using_deletions, label='using topmed deletions')
  plt.xlabel('Recall')
  plt.ylabel('Precision')
  plt.legend()
  fig = plt.gcf()
  fig.set_size_inches(10, 5)

  number_unconstrained_regions = df['truly constrained'].value_counts().loc[False]
  number_constrained_regions = df['truly constrained'].value_counts().loc[True]
  plt.title(
    'Predicting constraint on\n'
    f'{number_unconstrained_regions} unconstrained regions\n'
    f'and {number_constrained_regions} constrained regions'
  )
  
plot_precision_recall_wrapper(REGIONS)